In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
llm =  ChatOpenAI(model = 'gpt-3.5-turbo')

In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Attention.pdf")
docs = loader.load()



In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 200,chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
db = FAISS.from_documents(documents=documents[:20],embedding=OpenAIEmbeddings())

/var/folders/7y/t10qxx5s3dj1mj_3m8j2wqw00000gn/T/ipykernel_90157/3453760825.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(documents=documents[:20],embedding=OpenAIEmbeddings())


In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template('''
Answer the  following questions based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question:{input}''')


In [16]:
#chain creation
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt=prompt)


In [17]:
#Retreiver
retreiver = db.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x134fc0be0>, search_kwargs={})

In [18]:
#Retreiver chain 
from langchain.chains import create_retrieval_chain
retreiver_chain = create_retrieval_chain(retreiver,document_chain)


In [19]:
response = retreiver_chain.invoke({"input": "Who wrote the paper attention is all you need"})


In [20]:
response['answer']

'The paper "Attention Is All You Need" was written by Ashish Vaswani, Noam Shazeer, Niki Parmar, and Jakob Uszkoreit.'